# IFC PARSING [version. 170920]

## Require library

 - ifcopenshell
    
    http://ifcopenshell.org/python.html
    
    
 - OCC

    https://github.com/tpaviot/pythonocc-core/blob/master/INSTALL.md

###  libraries setting

In [2]:
import ifcopenshell
from ifcopenshell import geom
import os

In [3]:
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)

settings2 = ifcopenshell.geom.settings()

In [4]:
import numpy

In [5]:
import OCC.GProp
import OCC.BRepGProp

In [6]:
import operator

In [7]:
import pprint
import math

In [8]:
import OCCUtils

### IFC file set on
# ifcfileset

In [9]:
#occ_display = ifcopenshell.geom.utils.initialize_display()
# Open the IFC file using IfcOpenShell
ifc_file = ifcopenshell.open(os.path.join("D:\\projects\\capstone\\ifc_datas\\ifcs", "KIT_Institute_Arch.ifc"))
ifc_file_2 = ifcopenshell.open(os.path.join("D:\\projects\\capstone\\ifc_datas\\ifcs", "NBU_Duplex-Apt_Arch.ifc"))
ifc_file_3 = ifcopenshell.open(os.path.join("D:\\projects\\capstone\\ifc_datas\\ifcs", "1-r4b2.ifc"))
# The geometric elements in an IFC file are the IfcProduct elements. So these are 
# opened and displayed.


In [9]:
ifc_file

## Part of OCC

* targets
 * volume of each wall
 * area of each wall

### definition : helper function - OCC

In [10]:
# Some helper functions to map to the list of walls
def create_shape(elem):
    return ifcopenshell.geom.create_shape(settings, elem)

def calc_volume(s):
    props = OCC.GProp.GProp_GProps()
    OCC.BRepGProp.brepgprop_VolumeProperties(s.geometry, props)
    return props.Mass()
    
def calc_area(s):
    props = OCC.GProp.GProp_GProps()
    OCC.BRepGProp.brepgprop_SurfaceProperties(s.geometry, props)
    return props.Mass()
    
def normalize(li):
    mean, std = numpy.mean(li), numpy.std(li)
    return map(lambda v: abs(v-mean) / std, li)

def aboutelem(elem):
    return elem.wrapped_data

def get_geometry(s):
    return s.geometry

def create_shape2(elem):
    return ifcopenshell.geom.create_shape(settings2, elem)

def calc_centreofmass(s):
    prop = OCCUtils.Common.GpropsFromShape(s).surface()
    return prop.CentreOfMass().X(), prop.CentreOfMass().Y(), prop.CentreOfMass().Z(), prop.Mass()

def calc_staticmoments(s):
    g1 = OCC.GProp.GProp_GProps()
    OCC.BRepGProp.brepgprop_LinearProperties(s, g1)
    static_moments = g1.StaticMoments()
    print("static moments: {static_moments}".format(**vars()))
    return static_moments

### execution : OCC

In [49]:
# Obtain a list of walls from the model
walls = ifc_file.by_type("IfcWall")
# Create geometry for these walls
shapes = list(map(create_shape, walls))

print(len(walls))

for wall in walls:
    wall_shape = create_shape(wall)
    print(calc_centreofmass(wall_shape.geometry))

119
(9.17762998864916, 13.850000000000009, -1.7085105641982348, 92.29102597104443)
(32.822370011350856, 13.850000000000009, -1.7085105641982343, 92.29102597104443)
(41.85347629236344, 8.0, -1.6499999999999997, 72.49102597104442)
(0.14652370763657493, 8.000000000000002, -1.6499999999999997, 72.49102597104442)
(21.000000000000007, 2.1488673700482903, -1.690451069703692, 222.49102597104442)
(9.075016633399876, 9.150000000000007, -1.5494011976047917, 90.1799999999999)
(15.232239241429614, 6.849999999999999, -1.6040481400437634, 164.52000000000004)
(26.789499036608863, 9.150000000000002, -1.5528901734104041, 31.14)
(6.000000000000001, 4.5, -1.6500000000000001, 26.599999999999994)
(12.000000000000002, 4.5, -1.6500000000000001, 26.599999999999994)
(15.000000000000004, 11.500000000000002, -1.6500000000000001, 26.599999999999994)
(6.000000000000001, 11.500000000000002, -1.6500000000000001, 26.599999999999994)
(3.0000000000000004, 11.500000000000002, -1.6500000000000001, 26.599999999999994)
(9.0

In [71]:
# Obtain a list of walls from the model
walls = ifc_file.by_type("IfcWall")
# Create geometry for these walls
shapes = list(map(create_shape, walls))

print(len(walls))

for wall in walls:
    wall_shape = create_shape(wall)
    print(get_boundingbox(wall_shape.geometry))
    print("%f, %f, %f" % (center_boundingbox(wall_shape.geometry).Coord().X(), center_boundingbox(wall_shape.geometry).Coord().Y(), center_boundingbox(wall_shape.geometry).Coord().Z()))




119
(-0.00015, 13.69985, -3.00015, 18.300150000000002, 14.00015, -0.29984999999999984)
9.150000, 13.850000, -1.650000
(23.699849999999998, 13.69985, -3.00015, 42.00015, 14.00015, -0.29984999999999984)
32.850000, 13.850000, -1.650000
(41.6999, 1.9999, -3.0001, 42.0001, 14.0001, -0.29989999999999983)
41.850000, 8.000000, -1.650000
(-9.999999999977796e-05, 1.9999, -3.0001, 0.3001000000000006, 14.0001, -0.29989999999999983)
0.150000, 8.000000, -1.650000
(-0.00015000000000027754, 1.9998499999999992, -3.00015, 42.00015, 2.3001500000000004, -0.29984999999999984)
21.000000, 2.150000, -1.650000
(0.2998500000000007, 8.99985, -3.00015, 18.00015, 9.30015, -0.29984999999999984)
9.150000, 9.150000, -1.650000
(0.2998499999999972, 6.6998500000000005, -3.00015, 29.900150000000004, 7.00015, -0.29984999999999984)
15.100000, 6.850000, -1.650000
(23.99985, 8.99985, -3.00015, 29.90015, 9.30015, -0.29984999999999984)
26.950000, 9.150000, -1.650000
(5.8999, 2.2998999999999996, -3.0001, 6.1001, 6.7001, -0.2998

In [63]:
# Obtain a list of walls from the model
walls = ifc_file.by_type("IfcWall")
# Create geometry for these walls
shapes = list(map(create_shape, walls))

for wall in walls:
    wall_shape = create_shape(wall)
    print(wall_shape.data.geometry.brep_data)

       


CASCADE Topology V1, (c) Matra-Datavision
Locations 101
1
              1               0               0   2.4492127e-16 
              0               1               0              14 
              0               0               1              -3 
1
              1               0               0               0 
              0               1               0               0 
              0               0               1               0 
1
              1               0               0               0 
              0               1               0               0 
              0               0               1             2.7 
2  3 -1 0
2  3 1 2 1 0
1
              1               0               0               0 
              0               1               0               0 
              0               0               1             0.3 
1
             -1               0               0               0 
              0               0               1               0 


In [72]:
# Obtain a list of walls from the model
walls = ifc_file.by_type("IfcWall")
# Create geometry for these walls
shapes = list(map(create_shape, walls))

print(len(walls))

for wall in walls:
    wall_shape = create_shape(wall)
    pprint.pprint(wall_shape.data.product)


119
<bound method serialized_element_double_precision.product of <ifcopenshell.ifcopenshell_wrapper.serialized_element_double_precision; proxy of <Swig Object of type 'IfcGeom::SerializedElement< double > *' at 0x00000000161AA390> >>
<bound method serialized_element_double_precision.product of <ifcopenshell.ifcopenshell_wrapper.serialized_element_double_precision; proxy of <Swig Object of type 'IfcGeom::SerializedElement< double > *' at 0x00000000161AAD80> >>
<bound method serialized_element_double_precision.product of <ifcopenshell.ifcopenshell_wrapper.serialized_element_double_precision; proxy of <Swig Object of type 'IfcGeom::SerializedElement< double > *' at 0x00000000161AA4E0> >>
<bound method serialized_element_double_precision.product of <ifcopenshell.ifcopenshell_wrapper.serialized_element_double_precision; proxy of <Swig Object of type 'IfcGeom::SerializedElement< double > *' at 0x00000000161AA510> >>
<bound method serialized_element_double_precision.product of <ifcopenshell.i

In [69]:
# Obtain a list of walls from the model
walls = ifc_file.by_type("IfcWall")
# Create geometry for these walls
shapes = list(map(create_shape, walls))

print(len(walls))

for wall in walls:
    wall_shape = create_shape(wall)
    calc_staticmoments(wall_shape.geometry)


119
static moments: (3967.736129829718, 6063.448458813258, -641.7202857069958)
static moments: (14419.616597257487, 6063.448458813258, -641.7202857069958)
static moments: (4963.523610204754, 948.7529003975634, -195.6802857069975)
static moments: (17.429116882454345, 948.7529003975636, -195.6802857069975)
static moments: (16938.4763635436, 1733.817341981844, -1196.4802857070024)
static moments: (2613.4799999999987, 2569.3199999999993, -447.4799999999994)
static moments: (5245.600000000002, 2444.079999999999, -575.5199999999995)
static moments: (2986.359999999999, 1002.8399999999992, -175.56000000000012)
static moments: (350.39999999999986, 262.80000000000007, -96.35999999999997)
static moments: (700.8, 262.80000000000007, -96.35999999999997)
static moments: (876.0, 671.6000000000001, -96.35999999999997)
static moments: (350.39999999999986, 671.6000000000001, -96.35999999999997)
static moments: (175.2, 671.6000000000001, -96.35999999999997)
static moments: (525.5999999999999, 671.6000000

In [9]:
# Obtain a list of walls from the model
walls = ifc_file.by_type("IfcWall")
# Create geometry for these walls
shapes = list(map(create_shape, walls))
# Calculate their volumes
volumes = map(calc_volume, shapes)
# Calculate their surface areas
areas = map(calc_area, shapes)
# Compose a feature from the two measures
feature = normalize(map(operator.div, areas, volumes))

# Obtain a list of walls from the model
walls_st = ifc_file.by_type("IfcWallStandardCase")
# Create geometry for these walls
shapes_st = list(map(create_shape, walls))
# Calculate their volumes
volumes_st = map(calc_volume, shapes)
# Calculate their surface areas
areas_st = map(calc_area, shapes)
# Compose a feature from the two measures
feature_st = normalize(map(operator.div, areas, volumes))

print "areas : ", str(len(areas))
print "areas_st : ", str(len(areas_st))

areas :  119
areas_st :  119


## Part of ifcopenshell

* targets
 * locations of each wall
 * start and end point of each wall
 * axis of each wall

### IfcWallStatndardCase

In [10]:
# get walls location

walls = ifc_file.by_type("IfcWallStandardCase")
walls_info = []

loc_st = []

for wall in walls:
	gid = wall.get_info()['GlobalId']
	lid = wall.get_info()['id']

	location_info = wall.get_info()['ObjectPlacement'].get_info()['RelativePlacement'].get_info()
	loc_value = location_info['Location'].get_info()['Coordinates']
	loc_st.append(loc_value)
    
	if location_info['Axis'] == None:
		loc_axis = None
	else:
		loc_axis = location_info['Axis'].get_info()['DirectionRatios']

# 	print(gid, lid, loc_value, loc_axis)

len(loc_st)

119

In [17]:
loc_st[:5]

[(2.4492127e-16, 14.0, 0.0),
 (24.0, 14.0, 0.0),
 (42.0, 14.0, 0.0),
 (2.4492127e-16, 2.0, 0.0),
 (42.0, 2.0, 0.0)]

### IfcWall

In [19]:
# get walls location

walls = ifc_file.by_type("IfcWall")
walls_info = []

loc_a = []

for wall in walls:
	gid = wall.get_info()['GlobalId']
	lid = wall.get_info()['id']

	location_info = wall.get_info()['ObjectPlacement'].get_info()['RelativePlacement'].get_info()
	loc_value = location_info['Location'].get_info()['Coordinates']
	loc_a.append(loc_value)
    
	if location_info['Axis'] == None:
		loc_axis = None
	else:
		loc_axis = location_info['Axis'].get_info()['DirectionRatios']

# 	print(gid, lid, loc_value, loc_axis)

len(loc_a)

119

In [20]:
loc_a[:5]

[(2.4492127e-16, 14.0, 0.0),
 (24.0, 14.0, 0.0),
 (42.0, 14.0, 0.0),
 (2.4492127e-16, 2.0, 0.0),
 (42.0, 2.0, 0.0)]

### IfcDoor

In [42]:
# get walls location

doors = ifc_file.by_type("IfcDoor")
doors_info = []
doorslist = []

loc_d = []

for door in doors:
    gid = door.get_info()['GlobalId']
    lid = door.get_info()['id']

    location_info = door.get_info()['ObjectPlacement'].get_info()['RelativePlacement'].get_info()
    loc_value = location_info['Location'].get_info()['Coordinates']
    loc_d.append(loc_value)
    
    doorslist.append(door.get_info())
    
    if location_info['Axis'] == None:
        loc_axis = None
    else:
        loc_axis = location_info['Axis'].get_info()['DirectionRatios']

#     print(gid, lid, loc_value, loc_axis)

len(loc_d)
pprint.pprint(doorslist[:5])

[{'Description': None,
  'GlobalId': u'3qloCPnZDEQBom1ZjBlwJP',
  'Name': u'T\xfcr-014',
  'ObjectPlacement': #11251=IfcLocalPlacement(#11214,#11248),
  'ObjectType': None,
  'OverallHeight': 2.1,
  'OverallWidth': 1.0,
  'OwnerHistory': #29=IfcOwnerHistory(#28,#5,$,.NOCHANGE.,$,$,$,1215091190),
  'Representation': #11560=IfcProductDefinitionShape($,$,(#51125)),
  'Tag': u'1D1F256D-1AD5-42E5-82-9E-6EC603B6A7F1',
  'id': 11564,
  'type': 'IfcDoor'},
 {'Description': None,
  'GlobalId': u'0_flqizUv5_A_EGt7DzP3T',
  'Name': u'T\xfcr-015',
  'ObjectPlacement': #11695=IfcLocalPlacement(#11659,#11692),
  'ObjectType': None,
  'OverallHeight': 2.1,
  'OverallWidth': 1.0,
  'OwnerHistory': #29=IfcOwnerHistory(#28,#5,$,.NOCHANGE.,$,$,$,1215091190),
  'Representation': #12004=IfcProductDefinitionShape($,$,(#12152)),
  'Tag': u'FD98DDBA-05C3-4658-AC-8B-AFD0AFC31921',
  'id': 12008,
  'type': 'IfcDoor'},
 {'Description': None,
  'GlobalId': u'17n0YjYkvEshOmsrIPLGOb',
  'Name': u'T\xfcr-016',
  'Ob

In [23]:
loc_d[:5]

[(0.5, 0.215, 0.0),
 (0.5, 0.215, 0.0),
 (0.5, 0.215, 0.0),
 (0.5, 0.215, 0.0),
 (0.5, 0.215, 0.0)]

## Exporting data

* Datas
 * areas : area of each wall
 * volumes : volume of each wall
 * loc_a : locations of each wall

In [22]:
f = open('sample_loc.txt', 'w')

In [23]:
for i in range(len(areas)) :
    f.write(str(areas[i])+" "+str(volumes[i])+" "+str(loc_a[i][0])+" "+str(loc_a[i][1])+" "+str(loc_a[i][2]))
    f.write("\n")

### 유의미한 통계량 만들기

* Condition
 * 1 Architecture ~ 1 Data Set
 * Each data set should be equivalent to others.

* Version 1
 * mean and standard of wall datas distribution

#### Version 1 : simple wall datas distribution

In [27]:
area_mean = numpy.mean(areas)
area_std = numpy.std(areas)

volume_mean = numpy.mean(volumes)
volume_mean = numpy.std(volumes)

#loc_a는 데카트르 좌표에서 극좌표로 옮긴다음 두가지 양에 대해서 통계치 적용 mean std

51.193721322
45.5442874598


if both volumes and volumes_st exists,
... 디테일한 처리가 필요함

In [13]:
# totalVolumes = volumes+volumes_st
# totalAreas = areas+areas_st

In [13]:
for v in totalVolumes, totalAreas :
    print v

[11.459999999999996, 11.730000000000006, 9.718636363636247, 5.78999999999999, 4.3199999999999665, 18.779999999999756, 7.379999999999896, 4.656000000000115, 0.2999999999999804, 0.2954999999999822, 0.5973749999999989, 0.19200000000000653, 5.160000000000002, 3.24, 11.459999999999996, 11.730000000000006, 9.718636363636247, 5.78999999999999, 4.3199999999999665, 18.779999999999756, 7.379999999999896, 4.656000000000115, 0.2999999999999804, 0.2954999999999822, 0.5973749999999989, 0.19200000000000653, 5.160000000000002, 3.24]
[123.43999999999997, 124.47999999999999, 103.45291104588688, 62.959999999999866, 47.27999999999995, 198.07999999999993, 78.87999999999965, 50.464000000003566, 12.499999999999849, 12.316999999999908, 24.59324999999999, 8.107999999999892, 56.23999999999999, 35.75999999999999, 123.43999999999997, 124.47999999999999, 103.45291104588688, 62.959999999999866, 47.27999999999995, 198.07999999999993, 78.87999999999965, 50.464000000003566, 12.499999999999849, 12.316999999999908, 24.5

In [11]:
# print walls
# print shapes
print volumes
areas.sort()
areas.reverse()
print areas

# print feature

[11.459999999999996, 11.730000000000006, 9.718636363636247, 5.78999999999999, 4.3199999999999665, 18.779999999999756, 7.379999999999896, 4.656000000000115, 0.2999999999999804, 0.2954999999999822, 0.5973749999999989, 0.19200000000000653, 5.160000000000002, 3.24]
[198.07999999999993, 124.47999999999999, 123.43999999999997, 103.45291104588688, 78.87999999999965, 62.959999999999866, 56.23999999999999, 50.464000000003566, 47.27999999999995, 35.75999999999999, 24.59324999999999, 12.499999999999849, 12.316999999999908, 8.107999999999892]


### Area 를 ifcopenshell 로 얻어보자..

In [199]:
stdWalls = ifc_file.by_type("IfcWallStandardCase")
stdWalls_info = []

def extrudedArea_wall(tmp) :
    # ExtrudedSolid
    if tmp['type'] == 'IfcExtrudedAreaSolid' :
        if tmp['SweptArea'].get_info()['type'] == 'IfcArbitraryClosedProfileDef' :
            # 5 IfcCartesianPoint (last point equals to first point)
            return tmp['SweptArea'].get_info()['OuterCurve'].get_info()['Points']
        elif tmp['SweptArea'].get_info()['type'] == 'IfcRectangleProfileDef' :
            # x,y Coordinates
            return tmp['SweptArea'].get_info()['YDim'], tmp['SweptArea'].get_info()['XDim']
    else :
        return "not extruded"

def testtest(tmp) :
    # ExtrudedSolid
    if tmp['type'] == 'IfcExtrudedAreaSolid' :
        if tmp['SweptArea'].get_info()['type'] == 'IfcArbitraryClosedProfileDef' :
            # 5 IfcCartesianPoint (last point equals to first point)
            return tmp['Position'].get_info()
        elif tmp['SweptArea'].get_info()['type'] == 'IfcRectangleProfileDef' :
            # x,y Coordinates
            return tmp
    else :
        return "not extruded"
    
stdWall_i = 0
for stdWall in stdWalls:
    tmpdic = {}
#     print stdWall.get_info()
    #tmp = stdWall.get_info()['Representation'].get_info()['Representations'][0].get_info()['Items'][0].get_info()
    tmp = stdWall.get_info()['Representation'].get_info()['Representations'][1].get_info()['Items'][0].get_info()
    tmps = stdWall.get_info()['ObjectPlacement'].get_info()
#     tmpdic['tmp'] = tmp
    tmpdic['extrudedArea'] = extrudedArea_wall(tmp)
    tmpdic['test'] = testtest(tmp)
    tmpdic['placement'] = tmps
    tmpdic['guid'] = stdWall.GlobalId
    stdWalls_info.append(tmpdic)

#     # Clipping
#     clipping_i = 0
#     while tmp['type'] == 'IfcBooleanClippingResult' :
# #         print tmp['Operator'], tmp['FirstOperand'], tmp['SecondOperand']
            
#         if isinstance(tmp['SecondOperand'], ifcopenshell.entity_instance) :
        
#             tmp_s = tmp['SecondOperand'].get_info()
#             if tmp_s['type'] == 'IfcPolygonalBoundedHalfSpace' :
#                 # 5 IfcCartesianPoint (last point equals to first point)
#                 stdWalls_info.append("* IfcPolygonalBoundedHalfSpace *")
#                 print tmp_s
#                 stdWalls_info.append(tmp_s['PolygonalBoundary'].get_info()['Points'])
#             elif tmp_s['type'] == 'IfcHalfSpaceSolid' :
#                 stdWalls_info.append("* IfcHalfSpaceSolid pass *")
            
#             tmp = tmp['FirstOperand'].get_info()
            
#             stdWalls_info.append("* ExtrudedArea in Clipping *"+str(clipping_i))
#             stdWalls_info.append(extrudedArea_wall(tmp))
        
#         clipping_i += 1
    
    stdWall_i += 1

pprint.pprint(stdWalls_info)
#         if tmp['FirstOperand'].get_info()['type'] == 'IfcExtrudedAreaSolid' :
#             print tmp['FirstOperand'].get_info()['type']
#             if tmp['SweptArea'].get_info()['type'] == 'IfcArbitraryClosedProfileDef' :
#                 # 5 IfcCartesianPoint (last point equals to first point)
#                 print tmp['SweptArea'].get_info()['OuterCurve'].get_info()['Points']
#             if tmp['SweptArea'].get_info()['type'] == 'IfcRectangleProfileDef' :
#                 # x,y Coordinates
#                 print tmp['SweptArea'].get_info()['YDim'], tmp['SweptArea'].get_info()['XDim']
            
    # print tmp

    
    
# 	if tmp['type'] == "IfcBooleanClippingResult":
		
# 		while isinstance(tmp['FirstOperand'], ifcopenshell.entity_instance):
# 			# 5 CartesianPoint
# 			print tmp['SecondOperand'].get_info()['PolygonalBoundary'].get_info()
# 			tmp = tmp['FirstOperand'].get_info()
# 			if tmp['type'] == "IfcExtrudedAreaSolid":
# 				# 4 CartesianPoint
# 				print tmp['SweptArea'].get_info()['OuterCurve'].get_info()
# 				break

		

# 	if tmp['type'] == "IfcExtrudedAreaSolid":
# 		# 4 CartesianPoint
# 		tmp['SweptArea'].get_info()['OuterCurve'].get_info()

[{'extrudedArea': (#3045=IfcCartesianPoint((0.3,-0.3)),
                   #3049=IfcCartesianPoint((18.3,-0.3)),
                   #3054=IfcCartesianPoint((18.,0.)),
                   #3058=IfcCartesianPoint((-2.4492127E-16,0.)),
                   #3045=IfcCartesianPoint((0.3,-0.3))),
  'guid': u'2UZhzWc8v0fA0sfv__WgLn',
  'placement': {'PlacementRelTo': #2946=IfcLocalPlacement(#2926,#2943),
                'RelativePlacement': #3002=IfcAxis2Placement3D(#2998,#52,#44),
                'id': 3005,
                'type': 'IfcLocalPlacement'},
  'test': {'Axis': #52=IfcDirection((0.,0.,1.)),
           'Location': #56=IfcCartesianPoint((0.,0.,0.)),
           'RefDirection': #44=IfcDirection((1.,0.,0.)),
           'id': 3067,
           'type': 'IfcAxis2Placement3D'}},
 {'extrudedArea': (#5709=IfcCartesianPoint((-0.3,-0.3)),
                   #5713=IfcCartesianPoint((17.7,-0.3)),
                   #5717=IfcCartesianPoint((18.,0.)),
                   #5721=IfcCartesianPoint((0.,0.)

In [24]:
spaces = ifc_file.by_type("Ifcspace")
spaces_info = []

for space in spaces:
	tmp = space.get_info()['Representation'].get_info()['Representations'][0].get_info()['Items'][0].get_info()

	if tmp['type'] == "IfcBooleanClippingResult":
		
		while isinstance(tmp['FirstOperand'], ifcopenshell.entity_instance):
			# 5 CartesianPoint
			print tmp['SecondOperand'].get_info()['PolygonalBoundary'].get_info()
			tmp = tmp['FirstOperand'].get_info()
			if tmp['type'] == "IfcExtrudedAreaSolid":
				# 4 CartesianPoint
				print tmp['SweptArea'].get_info()['OuterCurve'].get_info()
				break

		

	if tmp['type'] == "IfcExtrudedAreaSolid":
		# 4 CartesianPoint
		tmp['SweptArea'].get_info()['OuterCurve'].get_info()

In [66]:
qareas = ifc_file.by_type("IfcQuantityArea")
qarea_info = []

for qarea in qareas:
	print qarea
	

In [65]:
elequants = ifc_file.by_type("IfcElementQuantity")
elequant_info = []

for elequant in elequants:
	print elequant.get_info()
	

### Ifc location 값 얻기

#### 1. 우선 좌표계를 생각 않고 좌표만 얻어와서 확인해본다.

* entity의 ObjectPlacement 속 Location 값

In [11]:
# get walls location

walls = ifc_file.by_type("IfcWallStandardCase")
walls_info = []

loc_st = []

for wall in walls:
	gid = wall.get_info()['GlobalId']
	lid = wall.get_info()['id']

	location_info = wall.get_info()['ObjectPlacement'].get_info()['RelativePlacement'].get_info()
	loc_value = location_info['Location'].get_info()['Coordinates']
	loc_st.append(loc_value)
    
	if location_info['Axis'] == None:
		loc_axis = None
	else:
		loc_axis = location_info['Axis'].get_info()['DirectionRatios']

# 	print(gid, lid, loc_value, loc_axis)

pprint.pprint(loc_st)

[(2.4492127e-16, 14.0, 0.0),
 (24.0, 14.0, 0.0),
 (42.0, 14.0, 0.0),
 (2.4492127e-16, 2.0, 0.0),
 (42.0, 2.0, 0.0),
 (18.0, 9.0, 0.0),
 (2.4492127e-16, 7.0, 0.0),
 (30.0, 9.0, 0.0),
 (6.0, 7.0, 0.0),
 (12.0, 7.0, 0.0),
 (15.0, 14.0, 0.0),
 (6.0, 14.0, 0.0),
 (3.0, 14.0, 0.0),
 (9.0, 14.0, 0.0),
 (12.0, 14.0, 0.0),
 (27.0, 14.0, 0.0),
 (18.0, 7.0, 0.0),
 (24.0, 7.0, 0.0),
 (30.0, 14.0, 0.0),
 (18.0, 16.0, 0.0),
 (18.0, 7.0, 0.0),
 (18.0, 14.0, 0.0),
 (24.0, 16.0, 0.0),
 (24.0, 14.0, 0.0),
 (2.4492127e-16, 2.0, 0.0),
 (42.0, 14.0, 0.0),
 (18.0, 2.0, 0.0),
 (18.0, 0.0, 0.0),
 (2.4492127e-16, 14.0, 0.0),
 (24.0, 14.0, 0.0),
 (18.0, 16.0, 0.0),
 (2.4492127e-16, 7.0, 0.0),
 (18.0, 9.0, 0.0),
 (42.0, 9.0, 0.0),
 (24.0, 7.0, 0.0),
 (27.0, 14.0, 0.0),
 (30.0, 14.0, 0.0),
 (33.0, 14.0, 0.0),
 (36.0, 14.0, 0.0),
 (39.0, 14.0, 0.0),
 (33.0, 7.0, 0.0),
 (9.0, 7.0, 0.0),
 (15.0, 7.0, 0.0),
 (15.0, 14.0, 0.0),
 (6.0, 14.0, 0.0),
 (3.0, 7.0, 0.0),
 (42.0, 2.0, 0.0),
 (18.0, 14.0, 0.0),
 (18.0, 2.0, 0.

* entity의 Representation 에서 위치를 찾아본다

In [13]:
stdWalls = ifc_file.by_type("IfcWallStandardCase")
stdWalls_info = []

def extrudedArea_wall(tmp) :
    # ExtrudedSolid
    if tmp['type'] == 'IfcExtrudedAreaSolid' :
        if tmp['SweptArea'].get_info()['type'] == 'IfcArbitraryClosedProfileDef' :
            # 5 IfcCartesianPoint (last point equals to first point)
            return tmp['SweptArea'].get_info()['OuterCurve'].get_info()['Points']
        elif tmp['SweptArea'].get_info()['type'] == 'IfcRectangleProfileDef' :
            # x,y Coordinates
            return tmp['SweptArea'].get_info()['YDim'], tmp['SweptArea'].get_info()['XDim']
    else :
        return "not extruded"

stdWall_i = 0
for stdWall in stdWalls:
#     print stdWall.get_info()
    #tmp = stdWall.get_info()['Representation'].get_info()['Representations'][0].get_info()['Items'][0].get_info()
    tmp = stdWall.get_info()['Representation'].get_info()['Representations'][1].get_info()['Items'][0].get_info()
    stdWalls_info.append(stdWall_i)
    stdWalls_info.append(tmp)
    stdWalls_info.append(extrudedArea_wall(tmp))
    
    stdWall_i += 1

pprint.pprint(stdWalls_info)

[0,
 {'Depth': 2.7,
  'ExtrudedDirection': #52=IfcDirection((0.,0.,1.)),
  'Position': #3067=IfcAxis2Placement3D(#56,#52,#44),
  'SweptArea': #3066=IfcArbitraryClosedProfileDef(.AREA.,$,#3062),
  'id': 3070,
  'type': 'IfcExtrudedAreaSolid'},
 (#3045=IfcCartesianPoint((0.3,-0.3)),
  #3049=IfcCartesianPoint((18.3,-0.3)),
  #3054=IfcCartesianPoint((18.,0.)),
  #3058=IfcCartesianPoint((-2.4492127E-16,0.)),
  #3045=IfcCartesianPoint((0.3,-0.3))),
 1,
 {'Depth': 2.7,
  'ExtrudedDirection': #52=IfcDirection((0.,0.,1.)),
  'Position': #5730=IfcAxis2Placement3D(#56,#52,#44),
  'SweptArea': #5729=IfcArbitraryClosedProfileDef(.AREA.,$,#5725),
  'id': 5733,
  'type': 'IfcExtrudedAreaSolid'},
 (#5709=IfcCartesianPoint((-0.3,-0.3)),
  #5713=IfcCartesianPoint((17.7,-0.3)),
  #5717=IfcCartesianPoint((18.,0.)),
  #5721=IfcCartesianPoint((0.,0.)),
  #5709=IfcCartesianPoint((-0.3,-0.3))),
 2,
 {'Depth': 2.7,
  'ExtrudedDirection': #52=IfcDirection((0.,0.,1.)),
  'Position': #7652=IfcAxis2Placement3D(#56

http://www.buildingsmart-tech.org/ifc/IFC2x3/TC1/html/ifcsharedbldgelements/lexical/ifcwallstandardcase.htm

Axis: A two-dimensional open curve (IfcBoundedCurve) defining the axis for the standard wall. The material layer offset is measured from the wall axis.
Body: A Swept Solid Representation or a CSG representation defining the 3D shape of the standard wall



####  Axis를 확인해보자


In [27]:
stdWalls = ifc_file.by_type("IfcWallStandardCase")
stdWalls_info = []

stdWall_i = 0
for stdWall in stdWalls:
#     print stdWall.get_info()
    #tmp = stdWall.get_info()['Representation'].get_info()['Representations'][0].get_info()['Items'][0].get_info()
    tmp = stdWall.get_info()['Representation'].get_info()['Representations'][0].get_info()['Items'][0].get_info()
    stdWall_dic = {}
    stdWall_dic[stdWall_i] = tmp
    stdWalls_info.append(stdWall_dic)
    
    stdWall_i += 1

pprint.pprint(stdWalls_info)

[{0: {'Points': (#3027=IfcCartesianPoint((0.,0.)),
                 #3031=IfcCartesianPoint((18.,0.))),
      'id': 3035,
      'type': 'IfcPolyline'}},
 {1: {'Points': (#5691=IfcCartesianPoint((0.,0.)),
                 #5695=IfcCartesianPoint((18.,0.))),
      'id': 5699,
      'type': 'IfcPolyline'}},
 {2: {'Points': (#7613=IfcCartesianPoint((0.,0.)),
                 #7617=IfcCartesianPoint((12.,0.))),
      'id': 7621,
      'type': 'IfcPolyline'}},
 {3: {'Points': (#7781=IfcCartesianPoint((0.,0.)),
                 #7785=IfcCartesianPoint((12.,0.))),
      'id': 7790,
      'type': 'IfcPolyline'}},
 {4: {'Points': (#7954=IfcCartesianPoint((0.,0.)),
                 #7958=IfcCartesianPoint((42.,0.))),
      'id': 7962,
      'type': 'IfcPolyline'}},
 {5: {'Points': (#11118=IfcCartesianPoint((0.,0.)),
                 #11122=IfcCartesianPoint((18.,0.))),
      'id': 11126,
      'type': 'IfcPolyline'}},
 {6: {'Points': (#12784=IfcCartesianPoint((0.,0.)),
                 #12788=Ifc

In [13]:
stdWalls = ifc_file.by_type("IfcWallStandardCase")
stdWalls_info = []

stdWall_i = 0
for stdWall in stdWalls:
#     print stdWall.get_info()
#     tmp = stdWall.get_info()['Representation'].get_info()
#     tmp = stdWall.get_info()['Representation'].get_info()['Representations'][0].get_info()
    tmp = stdWall.get_info()['Representation'].get_info()['Representations'][0].get_info()['Items'][0].get_info()
    #tmp = stdWall.get_info()['Representation'].get_info()['Representations'][0].get_info()['Items'][0].get_info()['Points'][1].get_info()
    stdWall_dic = {}
    stdWall_dic[stdWall_i] = tmp
    stdWalls_info.append(stdWall_dic)
    
    stdWall_i += 1

pprint.pprint(stdWalls_info)

[{0: {'Points': (#3027=IfcCartesianPoint((0.,0.)),
                 #3031=IfcCartesianPoint((18.,0.))),
      'id': 3035,
      'type': 'IfcPolyline'}},
 {1: {'Points': (#5691=IfcCartesianPoint((0.,0.)),
                 #5695=IfcCartesianPoint((18.,0.))),
      'id': 5699,
      'type': 'IfcPolyline'}},
 {2: {'Points': (#7613=IfcCartesianPoint((0.,0.)),
                 #7617=IfcCartesianPoint((12.,0.))),
      'id': 7621,
      'type': 'IfcPolyline'}},
 {3: {'Points': (#7781=IfcCartesianPoint((0.,0.)),
                 #7785=IfcCartesianPoint((12.,0.))),
      'id': 7790,
      'type': 'IfcPolyline'}},
 {4: {'Points': (#7954=IfcCartesianPoint((0.,0.)),
                 #7958=IfcCartesianPoint((42.,0.))),
      'id': 7962,
      'type': 'IfcPolyline'}},
 {5: {'Points': (#11118=IfcCartesianPoint((0.,0.)),
                 #11122=IfcCartesianPoint((18.,0.))),
      'id': 11126,
      'type': 'IfcPolyline'}},
 {6: {'Points': (#12784=IfcCartesianPoint((0.,0.)),
                 #12788=Ifc

In [32]:
stdWalls = ifc_file.by_type("IfcWallStandardCase")
stdWalls_info = []

stdWall_i = 0
for stdWall in stdWalls:
#     print stdWall.get_info()
#     tmp = stdWall.get_info()['Representation'].get_info()
#     tmp = stdWall.get_info()['Representation'].get_info()['Representations'][1].get_info()
#     tmp = stdWall.get_info()['Representation'].get_info()['Representations'][1].get_info()['Items'][0].get_info()
    tmp = stdWall.get_info()['Representation'].get_info()['Representations'][1].get_info()['Items'][0].get_info()
    stdWall_dic = {}
    stdWall_dic[stdWall_i] = tmp
    stdWalls_info.append(stdWall_dic)
    
    stdWall_i += 1

# pprint.pprint(stdWalls_info)

{'Axis': #52=IfcDirection((0.,0.,1.)),
 'Location': #56=IfcCartesianPoint((0.,0.,0.)),
 'RefDirection': #44=IfcDirection((1.,0.,0.)),
 'id': 3067,
 'type': 'IfcAxis2Placement3D'}
{'Axis': #52=IfcDirection((0.,0.,1.)),
 'Location': #56=IfcCartesianPoint((0.,0.,0.)),
 'RefDirection': #44=IfcDirection((1.,0.,0.)),
 'id': 5730,
 'type': 'IfcAxis2Placement3D'}
{'Axis': #52=IfcDirection((0.,0.,1.)),
 'Location': #56=IfcCartesianPoint((0.,0.,0.)),
 'RefDirection': #44=IfcDirection((1.,0.,0.)),
 'id': 7652,
 'type': 'IfcAxis2Placement3D'}
{'Axis': #52=IfcDirection((0.,0.,1.)),
 'Location': #56=IfcCartesianPoint((0.,0.,0.)),
 'RefDirection': #44=IfcDirection((1.,0.,0.)),
 'id': 7821,
 'type': 'IfcAxis2Placement3D'}
{'Axis': #52=IfcDirection((0.,0.,1.)),
 'Location': #56=IfcCartesianPoint((0.,0.,0.)),
 'RefDirection': #44=IfcDirection((1.,0.,0.)),
 'id': 7993,
 'type': 'IfcAxis2Placement3D'}
{'Axis': #52=IfcDirection((0.,0.,1.)),
 'Location': #56=IfcCartesianPoint((0.,0.,0.)),
 'RefDirection': #

KeyError: 'Position'

axis의 polyline 값이 달라야 할 것을 예상
그러나 polyline points 좌표를 갖는 axis를 확인.

가설 : 층별로 같은 polyline이 있을 것이다.

In [48]:
# input :
# List representation 
def getWallAxis(representation):
    line = representation['Representations'][0].get_info()['Items'][0].get_info()
    return (line['type'], line['Points'])

#### 다음으로 Area를 확인해볼 수 있을 것

In [26]:
stdWalls = ifc_file.by_type("IfcWallStandardCase")
stdWalls_info_2 = []

stdWall_i = 0
for stdWall in stdWalls:
#     print stdWall.get_info()
    #tmp = stdWall.get_info()['Representation'].get_info()['Representations'][0].get_info()['Items'][0].get_info()
    tmp = stdWall.get_info()['Representation'].get_info()['Representations'][1].get_info()
    stdWall_dic = {}
    stdWall_dic[stdWall_i] = tmp
    stdWalls_info_2.append(stdWall_dic)
    
    stdWall_i += 1

pprint.pprint(stdWalls_info_2)

[{0: {'ContextOfItems': #67=IfcGeometricRepresentationContext('Plan','Design',3,1.0000000E-5,#60,#63),
      'Items': (#3070=IfcExtrudedAreaSolid(#3066,#3067,#52,2.7),),
      'RepresentationIdentifier': u'Body',
      'RepresentationType': u'SweptSolid',
      'id': 3073,
      'type': 'IfcShapeRepresentation'}},
 {1: {'ContextOfItems': #67=IfcGeometricRepresentationContext('Plan','Design',3,1.0000000E-5,#60,#63),
      'Items': (#5733=IfcExtrudedAreaSolid(#5729,#5730,#52,2.7),),
      'RepresentationIdentifier': u'Body',
      'RepresentationType': u'SweptSolid',
      'id': 5736,
      'type': 'IfcShapeRepresentation'}},
 {2: {'ContextOfItems': #67=IfcGeometricRepresentationContext('Plan','Design',3,1.0000000E-5,#60,#63),
      'Items': (#7655=IfcExtrudedAreaSolid(#7651,#7652,#52,2.7),),
      'RepresentationIdentifier': u'Body',
      'RepresentationType': u'SweptSolid',
      'id': 7658,
      'type': 'IfcShapeRepresentation'}},
 {3: {'ContextOfItems': #67=IfcGeometricRepresentati

### IfcRelDefinesByProperties

* 특징
 * 이 Entity가 없는 ifc 파일도 있을 수 있음
 * 좌표 정보 그러니까, Locations는 얻을 수 없음
 * IfcElementQuantity와 IfcPropertySet이 중요 Types.
 * IFcPropertySet는 자유롭게 정의하여 사용하는 Type, 따라서 없는 경우 있음.
 

In [43]:
relbyprops = ifc_file.by_type("IfcRelDefinesByProperties")
relbyprop_info = []


standardWall_n = 0

for relbyprop in relbyprops:
    RelPropDef = relbyprop.get_info()['RelatingPropertyDefinition'].get_info() # IfcPropertySet, IfcElementQuantity
    RelObj = relbyprop.get_info()['RelatedObjects'][0].get_info() # related object
    
    # IfcWallStandardCase
    
    if 'IfcWallStandardCase' == RelObj['type'] :
        
        if RelPropDef['type'] == 'IfcElementQuantity' :
            for k,v in enumerate(RelPropDef['Quantities']) :
                if v.get_info()['type'] == 'IfcQuantityLength' :
                    print v.get_info()['Name'], v.get_info()['LengthValue']
                if v.get_info()['type'] == 'IfcQuantityArea' :
                    print v.get_info()['Name'], v.get_info()['AreaValue']
                if v.get_info()['type'] == 'IfcQuantityVolume' :
                    print v.get_info()['Name'], v.get_info()['VolumeValue']
        if RelPropDef['type'] == 'IfcPropertySet' :
            print RelPropDef['HasProperties']
        standardWall_n += 1
    
print standardWall_n
	

(#5623=IfcPropertySingleValue('ThermalTransmittance',$,IfcThermalTransmittanceMeasure(0.3),$), #5628=IfcPropertySingleValue('IsExternal',$,IfcBoolean(.T.),$), #5633=IfcPropertySingleValue('LoadBearing',$,IfcBoolean(.T.),$))
Width 0.3
Height 2.7
Length 18.0
GrossSideArea 48.6
NetSideArea 30.6
GrossVolume 14.58
NetVolume 9.18
GrossFootprintArea 5.4
(#7541=IfcPropertySingleValue('ThermalTransmittance',$,IfcThermalTransmittanceMeasure(0.3),$), #7546=IfcPropertySingleValue('IsExternal',$,IfcBoolean(.T.),$), #7551=IfcPropertySingleValue('LoadBearing',$,IfcBoolean(.T.),$))
Width 0.3
Height 2.7
Length 18.0
GrossSideArea 48.6
NetSideArea 30.6
GrossVolume 14.58
NetVolume 9.18
GrossFootprintArea 5.4
(#7713=IfcPropertySingleValue('ThermalTransmittance',$,IfcThermalTransmittanceMeasure(0.3),$), #7718=IfcPropertySingleValue('IsExternal',$,IfcBoolean(.T.),$), #7723=IfcPropertySingleValue('LoadBearing',$,IfcBoolean(.T.),$))
Width 0.3
Height 2.7
Length 12.0
GrossSideArea 32.4
NetSideArea 32.4
GrossVolu

In [34]:
relbyprops = ifc_file.by_type("IfcRelDefinesByProperties")
relbyprop_info = []

standardWall_n = 0

for relbyprop in relbyprops:
    RelPropDef = relbyprop.get_info()['RelatingPropertyDefinition'].get_info() # IfcPropertySet, IfcElementQuantity
    RelObj = relbyprop.get_info()['RelatedObjects'][0].get_info() # related object
    
    # IfcWallStandardCase
    
    relbyprop_info.append(relbyprop.get_info())
    relbyprop_info.append(RelPropDef)
    relbyprop_info.append(RelObj)
    
pprint.pprint(relbyprop_info[:6])
	

[{'Description': None,
  'GlobalId': u'1iVk5gJTb5PhXiUuI2GcDy',
  'Name': None,
  'OwnerHistory': #29=IfcOwnerHistory(#28,#5,$,.NOCHANGE.,$,$,$,1215091190),
  'RelatedObjects': (#70=IfcProject('1dzyWEOh99jRUBVYP$exmF',#29,'Projekt Buerogebaeude','No real Project',$,$,'Concept',(#67,#229,#275),#42),),
  'RelatingPropertyDefinition': #77=IfcPropertySet('3j_km74hL9FBi0Nb34cXN$',#29,'Pset_ProjectCommon',$,(#83,#88,#93)),
  'id': 81,
  'type': 'IfcRelDefinesByProperties'},
 {'Description': None,
  'GlobalId': u'3j_km74hL9FBi0Nb34cXN$',
  'HasProperties': (#83=IfcPropertySingleValue('ConstructionMode',$,IfcLabel('Massivbau'),$),
                    #88=IfcPropertySingleValue('BuildingPermitId',$,IfcIdentifier('4711'),$),
                    #93=IfcPropertySingleValue('GrossAreaPlanned',$,IfcAreaMeasure(1000.),$)),
  'Name': u'Pset_ProjectCommon',
  'OwnerHistory': #29=IfcOwnerHistory(#28,#5,$,.NOCHANGE.,$,$,$,1215091190),
  'id': 77,
  'type': 'IfcPropertySet'},
 {'Description': u'No real Pr

#### 결론

* 상황
 * '벽'과 '벽'의 관계만이 아니라, '프로젝트'와 '프로젝트 특성' 관계도 정의됨
 * 또한 관계정보의 경향 확인해서 일반화해야 함
* 따라서, rule-based로 RelatedObjects 별로 처리해야 함

## 층 구별

### 먼저 층을 확인한다

In [37]:
bdstoreys = ifc_file.by_type("IfcBuildingStorey")
bdstoreys_info = []

bdstoreys_n = 0

for storey in bdstoreys:
    storeyinfo = storey.get_info()
    bdstoreys_info.append(storeyinfo)

pprint.pprint(bdstoreys_info)

[{'CompositionType': u'ELEMENT',
  'Description': u'Basement',
  'Elevation': -3.0,
  'GlobalId': u'25SMdCQszBi9al8gnrh8LV',
  'LongName': None,
  'Name': u'Keller',
  'ObjectPlacement': #2946=IfcLocalPlacement(#2926,#2943),
  'ObjectType': None,
  'OwnerHistory': #29=IfcOwnerHistory(#28,#5,$,.NOCHANGE.,$,$,$,1215091190),
  'Representation': None,
  'id': 2949,
  'type': 'IfcBuildingStorey'},
 {'CompositionType': u'ELEMENT',
  'Description': u'Ground Level',
  'Elevation': 0.0,
  'GlobalId': u'1n2Lw3QubAj8dNIkFMcbgd',
  'LongName': None,
  'Name': u'Erdgeschoss',
  'ObjectPlacement': #38833=IfcLocalPlacement(#2926,#38830),
  'ObjectType': None,
  'OwnerHistory': #29=IfcOwnerHistory(#28,#5,$,.NOCHANGE.,$,$,$,1215091190),
  'Representation': None,
  'id': 38836,
  'type': 'IfcBuildingStorey'},
 {'CompositionType': u'ELEMENT',
  'Description': u'1. Level',
  'Elevation': 3.0,
  'GlobalId': u'3edQNiDA9BMfnQgyxoNyj_',
  'LongName': None,
  'Name': u'1. Obergeschoss',
  'ObjectPlacement': #6

### 각 층에 있는 건축요소 즉, RelatedEelements 의 모음을 확인할 수 있다.

In [32]:
structures = ifc_file.by_type("IfcRelContainedInSpatialStructure")
con_structures_info = []

con_sturctures_n = 0

for st in structures:
    sta = st.get_info()
    con_structures_info.append(sta)

for storycontainer in con_structures_info :
    pprint.pprint(storycontainer['Description'])

u'BuildingStoreyContainer for Building Elements'
u'BuildingStoreyContainer for Building Elements'
u'BuildingStoreyContainer for Building Elements'
u'BuildingStoreyContainer for Building Elements'
u'BuildingStoreyContainer for Building Elements'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceContainer for Products'
u'SpaceCont

IfcRelContainedInSpatialStructure에는

['Description', 'GlobalId', 'OwnerHistory', 'RelatedElements', 'RelatingStructure', 'type', 'id', 'Name']

'BuildingStoreyContainer', 'SpaceContainer' 을 포함하여, 갖고 있음.

'BuildingStoreyContainer' 만 골라서.

In [174]:
# input :
# List representation 
def getWallAxis(representation):
    line = representation['Representations'][0].get_info()['Items'][0].get_info()
    return (line['type'], line['Points'])

In [ ]:
def testtest(tmp) :
    # ExtrudedSolid
    if tmp['type'] == 'IfcExtrudedAreaSolid' :
        if tmp['SweptArea'].get_info()['type'] == 'IfcArbitraryClosedProfileDef' :
            # 5 IfcCartesianPoint (last point equals to first point)
            return tmp['Position'].get_info()
        elif tmp['SweptArea'].get_info()['type'] == 'IfcRectangleProfileDef' :
            # x,y Coordinates
            return tmp
    else :
        return "not extruded"

# 찾았다. rp 부분의 Axis Location RefDirection으로 조정 하자.

In [224]:
structures = ifc_file.by_type("IfcRelContainedInSpatialStructure")
con_structures_info = []

for st in structures:
    sta = st.get_info()
    if 0 == (sta['Description'].find("BuildingStoreyContainer")):
        con_structures_info.append(sta)

stories = {}
walls = []
storey_index = 0

for storeycontainer in con_structures_info :
    walls = []
#     pprint.pprint(storeycontainer['Name'])
    for element in storeycontainer['RelatedElements']:
#         print(element.get_info()['type'])
        if "IfcWallStandardCase" == element.get_info()['type']:
            wall = {}
            line = getWallAxis(element.get_info()['Representation'].get_info())
            tmp = element.get_info()['Representation'].get_info()['Representations'][1].get_info()['Items'][0].get_info()
            tmp1 = element.get_info()['ObjectPlacement'].get_info()['RelativePlacement'].get_info()
            tmp2 = element.get_info()['ObjectPlacement'].get_info()['PlacementRelTo'].get_info()['RelativePlacement'].get_info()
            tmp3 = element.get_info()['ObjectPlacement'].get_info()['PlacementRelTo'].get_info()['PlacementRelTo'].get_info()['RelativePlacement'].get_info()
            wall['line'] = line
            wall['prt-rp'] = tmp2
            wall['rp'] = tmp1
            wall['prt-prt-rp'] = tmp3
            wall['guid'] = element.GlobalId
            walls.append(wall)
            
            pprint.pprint(wall['rp'])
            
    stories[storey_index] = walls
    storey_index = storey_index + 1
    
# pprint.pprint(stories)

{'Axis': #52=IfcDirection((0.,0.,1.)),
 'Location': #2998=IfcCartesianPoint((2.4492127E-16,14.,0.)),
 'RefDirection': #44=IfcDirection((1.,0.,0.)),
 'id': 3002,
 'type': 'IfcAxis2Placement3D'}
{'Axis': #52=IfcDirection((0.,0.,1.)),
 'Location': #5662=IfcCartesianPoint((24.,14.,0.)),
 'RefDirection': #44=IfcDirection((1.,0.,0.)),
 'id': 5666,
 'type': 'IfcAxis2Placement3D'}
{'Axis': #52=IfcDirection((0.,0.,1.)),
 'Location': #7584=IfcCartesianPoint((42.,14.,0.)),
 'RefDirection': #7580=IfcDirection((0.,-1.,0.)),
 'id': 7588,
 'type': 'IfcAxis2Placement3D'}
{'Axis': #52=IfcDirection((0.,0.,1.)),
 'Location': #7752=IfcCartesianPoint((2.4492127E-16,2.,0.)),
 'RefDirection': #48=IfcDirection((0.,1.,0.)),
 'id': 7756,
 'type': 'IfcAxis2Placement3D'}
{'Axis': #52=IfcDirection((0.,0.,1.)),
 'Location': #7925=IfcCartesianPoint((42.,2.,0.)),
 'RefDirection': #7921=IfcDirection((-1.,0.,0.)),
 'id': 7929,
 'type': 'IfcAxis2Placement3D'}
{'Axis': #52=IfcDirection((0.,0.,1.)),
 'Location': #11088=If

In [ ]:
structures = ifc_file.by_type("IfcRelContainedInSpatialStructure")
con_structures_info = []

for st in structures:
    sta = st.get_info()
    if 0 == (sta['Description'].find("BuildingStoreyContainer")):
        con_structures_info.append(sta)

stories = {}
walls = []
storey_index = 0

for storeycontainer in con_structures_info :
    walls = []
#     pprint.pprint(storeycontainer['Name'])
    for element in storeycontainer['RelatedElements']:
#         print(element.get_info()['type'])
        if "IfcWallStandardCase" == element.get_info()['type']:
            wall = {}
            line = getWallAxis(element.get_info()['Representation'].get_info())
            tmp = element.get_info()['Representation'].get_info()['Representations'][1].get_info()['Items'][0].get_info()
            tmp1 = element.get_info()['ObjectPlacement'].get_info()['RelativePlacement'].get_info()
            tmp2 = element.get_info()['ObjectPlacement'].get_info()['PlacementRelTo'].get_info()['RelativePlacement'].get_info()
            tmp3 = element.get_info()['ObjectPlacement'].get_info()['PlacementRelTo'].get_info()['PlacementRelTo'].get_info()['RelativePlacement'].get_info()
            wall['line'] = line
            wall['prt-rp'] = tmp2
            wall['rp'] = tmp1
            wall['prt-prt-rp'] = tmp3
            wall['guid'] = element.GlobalId
            walls.append(wall)
            
            pprint.pprint(wall['rp'])
            
    stories[storey_index] = walls
    storey_index = storey_index + 1
    
# pprint.pprint(stories)

# qwerty complete

In [13]:
structures = ifc_file_3.by_type("IfcRelContainedInSpatialStructure")
con_structures_info = []
con_structures_info_2 = []

# input :
# List representation 
def getWallAxis(representation):
    line = representation['Representations'][0].get_info()['Items'][0].get_info()
    return (line['type'], line['Points'])

# 선형변환 회전
# input :
# List refDirection
# 현재는 x, y 벡터에 대해서만 회전을 염두.
def rotationCoor(refDirection):
    theta = 0
    if refDirection[0] == 1.0 and refDirection[1] == 0.0 and refDirection[2] == 0.0 :
        theta = 0
    elif refDirection[0] == -1.0 and refDirection[1] == 0.0 and refDirection[2] == 0.0 :
        theta = numpy.pi
    elif refDirection[0] == 0.0 and refDirection[1] == 1.0 and refDirection[2] == 0.0 :
        theta = numpy.pi * 3 / 2
    elif refDirection[0] == 0.0 and refDirection[1] == -1.0 and refDirection[2] == 0.0 :
        theta = numpy.pi / 2
    return numpy.array([[math.cos(theta), (-1.0*math.sin(theta)), 0.0],
                    [math.sin(theta), math.cos(theta), 0.0],
                       [0.0, 0.0, 1.0]])
# 사이각 구하기
# input :
# List vectorA, List vectorB
# 현재는 vectorA, vectorB
def getRadianBetween_A_B(vectorA, vectorB): 
    va = numpy.array(vectorA)
    vb = numpy.array(vectorB)
    v0 = numpy.array([1.0, 0.0, 0.0])
    dp = numpy.dot(numpy.array(v0), (vb - va)) / (numpy.linalg.norm(v0) * numpy.linalg.norm(vb - va))
    return numpy.arccos(dp)


# 길이 구하기
# input :
# List vectorA, List vectorB
# 현재는
def getLengthBetween_A_B(vectorA, vectorB):
    return numpy.linalg.norm(numpy.array(vectorA) - numpy.array(vectorB))

for st in structures:
    sta = st.get_info()
    if sta['RelatingStructure'].get_info()['type'] == "IfcBuildingStorey":
        con_structures_info.append(sta)
#         pprint.pprint(sta['RelatingStructure'].get_info()['ObjectPlacement'].get_info()['RelativePlacement'].get_info())
#         pprint.pprint(sta['RelatingStructure'].get_info()['ObjectPlacement'].get_info()['PlacementRelTo'].get_info())
    elif sta['RelatingStructure'].get_info()['type'] == "IfcSpace" :
        con_structures_info_2.append(sta)
        
stories = []
spaces = []

for storeycontainer in con_structures_info :
    walls = []
    i = 0
    for element in storeycontainer['RelatedElements']:
#         print(element.get_info()['type'])
        if "IfcWallStandardCase" == element.get_info()['type']:
#         if "IfcWall" == element.get_info()['type']:
            i = i + 1
            wall = {}
            line = getWallAxis(element.get_info()['Representation'].get_info())
            wall['startpoint'] = line[1][0].get_info()['Coordinates']
            wall['endpoint'] = line[1][1].get_info()['Coordinates']
            wall['guid'] = element.GlobalId
            tmp1 = element.get_info()['ObjectPlacement'].get_info()['RelativePlacement'].get_info()
#             tmp2 = element.get_info()['ObjectPlacement'].get_info()['PlacementRelTo'].get_info()['PlacementRelTo'].get_info()['PlacementRelTo'].get_info()['RelativePlacement'].get_info()
#             pprint.pprint(tmp2)
            if tmp1['Axis'] == None:
                wall['z-axis'] = [0.0, 0.0, 1.0]
            else :
                wall['z-axis'] = tmp1['Axis'].get_info()
            if tmp1['RefDirection'] == None:
                wall['x-axis'] = [1.0, 0.0, 0.0]
            else :
                wall['x-axis'] = [tmp1['RefDirection'].get_info()['DirectionRatios'][0],
                                 tmp1['RefDirection'].get_info()['DirectionRatios'][1],
                                 tmp1['RefDirection'].get_info()['DirectionRatios'][2]]
#             pprint.pprint(element.get_info()['ObjectPlacement'].get_info()['PlacementRelTo'].get_info()['RelativePlacement'])
            wall['location'] = tmp1['Location'].get_info()
            walls.append(wall)
    print(i)
    stories.append(walls)

for spacecontainer in con_structures_info_2 :
    walls = []
#     pprint.pprint(storeycontainer['Name'])
    for element in spacecontainer['RelatedElements']:
#         print(element.get_info()['type'])
#         if "IfcWallStandardCase" == element.get_info()['type']:
        if "IfcWallStandardCase" == element.get_info()['type']:
            wall = {}
            line = getWallAxis(element.get_info()['Representation'].get_info())
            wall['startpoint'] = line[1][0].get_info()['Coordinates']
            wall['endpoint'] = line[1][1].get_info()['Coordinates']
            wall['guid'] = element.GlobalId
            tmp1 = element.get_info()['ObjectPlacement'].get_info()['RelativePlacement'].get_info()
#             tmp2 = element.get_info()['ObjectPlacement'].get_info()['PlacementRelTo'].get_info()['PlacementRelTo'].get_info()['PlacementRelTo'].get_info()['RelativePlacement'].get_info()
#             pprint.pprint(tmp2)
            if tmp1['Axis'] == None:
                wall['z-axis'] = [0.0, 0.0, 1.0]
            else :
                wall['z-axis'] = tmp1['Axis'].get_info()
            if tmp1['RefDirection'] == None:
                wall['x-axis'] = [1.0, 0.0, 0.0]
            else :
                wall['x-axis'] = [tmp1['RefDirection'].get_info()['DirectionRatios'][0],
                                 tmp1['RefDirection'].get_info()['DirectionRatios'][1],
                                 tmp1['RefDirection'].get_info()['DirectionRatios'][2]]
#             pprint.pprint(element.get_info()['ObjectPlacement'].get_info()['PlacementRelTo'].get_info()['RelativePlacement'])
            wall['location'] = tmp1['Location'].get_info()
            walls.append(wall)
    spaces.append(walls)
    
walls_stories_data = []
for walls_storey in stories:
    walls_storey_data = []
    for line in walls_storey:
        line_data = []
        rotationMat = rotationCoor(line['x-axis'])
        np_pnt_0 = numpy.zeros((1,3))
        np_pnt_0 = numpy.array([[line['startpoint'][0], line['startpoint'][1], 0.0]])
        np_pnt_1 = numpy.zeros((1,3))
        np_pnt_1 = numpy.array([[line['endpoint'][0], line['endpoint'][1], 0.0]])
        
        new_vert_0 = numpy.dot(np_pnt_0, rotationMat).tolist()[0]
        new_vert_1 = numpy.dot(np_pnt_1, rotationMat).tolist()[0]
    
        vert_0 = []
        vert_1 = []
        
        for i in range(0,3):
            vert_0.append(new_vert_0[i] + line['location']['Coordinates'][i])
            vert_1.append(new_vert_1[i] + line['location']['Coordinates'][i])
        
        line_data = [vert_0, vert_1]
        walls_storey_data.append(line_data)
        
#         pprint.pprint(new_vert_0)
#         pprint.pprint(new_vert_1)
#         pprint.pprint(np_pnt_1.tolist())
        
    walls_stories_data.append(walls_storey_data)

    
walls_spaces_data = []
for walls_space in spaces:
    walls_space_data = []
    for line in walls_space:
        line_data = []
        rotationMat = rotationCoor(line['x-axis'])
        np_pnt_0 = numpy.zeros((1,3))
        np_pnt_0 = numpy.array([[line['startpoint'][0], line['startpoint'][1], 0.0]])
        np_pnt_1 = numpy.zeros((1,3))
        np_pnt_1 = numpy.array([[line['endpoint'][0], line['endpoint'][1], 0.0]])
        
        new_vert_0 = numpy.dot(np_pnt_0, rotationMat).tolist()[0]
        new_vert_1 = numpy.dot(np_pnt_1, rotationMat).tolist()[0]
    
        vert_0 = []
        vert_1 = []
        
        for i in range(0,3):
            vert_0.append(new_vert_0[i] + line['location']['Coordinates'][i])
            vert_1.append(new_vert_1[i] + line['location']['Coordinates'][i])
        
        line_data = [vert_0, vert_1]
        walls_space_data.append(line_data)
        
#         pprint.pprint(new_vert_0)
#         pprint.pprint(new_vert_1)
#         pprint.pprint(np_pnt_1.tolist())
        
    walls_spaces_data.append(walls_space_data)

def getMinValue(walls_stories_data):
    point_x_list = []
    point_y_list = []
    for wall in walls_stories_data[0]:
        for point in wall:
            point_x_list.append(point[0])
            point_y_list.append(point[1])
    
    x_min_value = min(point_x_list)
    y_min_value = min(point_y_list)
    
    return (x_min_value, y_min_value)

def getData_nn(walls_stories_data):
    newWalls = []
    newWalls_rad = []
    newWalls_len = []
    for wall in walls_stories_data[0]:
        newWall = []
        for point in wall:
            newPoint = []
            newPoint.append(point[0] - getMinValue(walls_stories_data)[0])
            newPoint.append(point[1] - getMinValue(walls_stories_data)[1])
            newPoint.append(0.0)
            newWall.append(newPoint)
        newWalls_len.append(getLengthBetween_A_B(newWall[0], newWall[1]))
        newWalls_rad.append(getRadianBetween_A_B(newWall[0], newWall[1]))
        newWalls.append(newWall)

    data = {}
    data['vert'] = newWalls
    data['length'] = newWalls_len
    data['radian'] = newWalls_rad

    data_nn = []
    for i in range(len(newWalls_len)):
        data_nn.append([ data['vert'][i][0][0], data['vert'][i][0][1], data['vert'][i][0][2],
        data['vert'][i][1][0], data['vert'][i][1][1], data['vert'][i][1][2], data['length'][i], data['radian'][i]])
    return data_nn

def flipWalls(walls_stories_data, flip_index):
    flipwalls_list = []
    if flip_index == 0:
        return walls_stories_data
    elif flip_index == 1:
        newWalls = []
        for wall in walls_stories_data[0]:
            newPoint_0 = []
            newPoint_1 = []
            newPoint_0 = [wall[0][0], -1*wall[0][1], wall[0][2]]
            newPoint_1 = [wall[0][0], -1*wall[0][1], wall[0][2]]
            newWalls.append([newPoint_0, newPoint_1])
        flipwalls_list.append(newWalls)
        return flipwalls_list

def rotateWalls(walls_stories_data, rotate_index):
    rotateWalls_list = []
    if rotate_index == 0:
        return walls_stories_data
    elif rotate_index == 1:
        rotationMat = rotationCoor([-1.0, 0.0, 0.0])
        story = []
        for wall in walls_stories_data[0]:
            np_pnt_0 = numpy.zeros((1,3))
            np_pnt_0 = numpy.array([[wall[0][0], wall[0][1], 0.0]])
            np_pnt_1 = numpy.zeros((1,3))
            np_pnt_1 = numpy.array([[wall[1][0], wall[1][1], 0.0]])

            new_vert_0 = numpy.dot(np_pnt_0, rotationMat).tolist()[0]
            new_vert_1 = numpy.dot(np_pnt_1, rotationMat).tolist()[0]
            story.append([new_vert_0, new_vert_1])
        rotateWalls_list.append(story)
        return rotateWalls_list
    elif rotate_index == 2:
        rotationMat = rotationCoor([0.0, 1.0, 0.0])
        story = []
        for wall in walls_stories_data[0]:
            np_pnt_0 = numpy.zeros((1,3))
            np_pnt_0 = numpy.array([[wall[0][0], wall[0][1], 0.0]])
            np_pnt_1 = numpy.zeros((1,3))
            np_pnt_1 = numpy.array([[wall[1][0], wall[1][1], 0.0]])

            new_vert_0 = numpy.dot(np_pnt_0, rotationMat).tolist()[0]
            new_vert_1 = numpy.dot(np_pnt_1, rotationMat).tolist()[0]
            story.append([new_vert_0, new_vert_1])
        rotateWalls_list.append(story)
        return rotateWalls_list
    elif rotate_index == 3:
        rotationMat = rotationCoor([0.0, -1.0, 0.0])
        story = []
        for wall in walls_stories_data[0]:
            np_pnt_0 = numpy.zeros((1,3))
            np_pnt_0 = numpy.array([[wall[0][0], wall[0][1], 0.0]])
            np_pnt_1 = numpy.zeros((1,3))
            np_pnt_1 = numpy.array([[wall[1][0], wall[1][1], 0.0]])

            new_vert_0 = numpy.dot(np_pnt_0, rotationMat).tolist()[0]
            new_vert_1 = numpy.dot(np_pnt_1, rotationMat).tolist()[0]
            story.append([new_vert_0, new_vert_1])
        rotateWalls_list.append(story)
        return rotateWalls_list

def getVariations(walls_stories_data):
    story_walls_data = []
    for i in range(4):
        for j in range(2):
            story_walls_data.append(flipWalls(rotateWalls(walls_stories_data, i), j))
    return story_walls_data


point_x_list = []
point_y_list = []
for wall in walls_stories_data[0]:
    for point in wall:
        point_x_list.append(point[0])
        point_y_list.append(point[1])

x_min_value = min(point_x_list)
y_min_value = min(point_y_list)

newWalls = []
newWalls_rad = []
newWalls_len = []
for wall in walls_stories_data[0]:
    newWall = []
    for point in wall:
        newPoint = []
        newPoint.append(point[0] - x_min_value)
        newPoint.append(point[1] - y_min_value)
        newPoint.append(0.0)
        newWall.append(newPoint)
    newWalls_len.append(getLengthBetween_A_B(newWall[0], newWall[1]))
    newWalls_rad.append(getRadianBetween_A_B(newWall[0], newWall[1]))
    newWalls.append(newWall)

data = {}
data['vert'] = newWalls
data['length'] = newWalls_len
data['radian'] = newWalls_rad

data_nn = []
for i in range(len(newWalls_len)):
    data_nn.append([ data['vert'][i][0][0], data['vert'][i][0][1], data['vert'][i][0][2],
    data['vert'][i][1][0], data['vert'][i][1][1], data['vert'][i][1][2], data['length'][i], data['radian'][i]])

    
def wallsort(wall):
    return (((wall[0]+wall[3])/2)**2+((wall[1]+wall[4])/2)**2)**(0.5)

data_sorted = sorted(data_nn, key=wallsort)


def wallsort_chain(walls):

    sorted_walls = []
    temp_index = []
    
    for i in range(0, len(walls)):
        temp_1 = []
        temp_2 = []
        if i == 0:
            sorted_walls.append(walls[i])
            continue
        else:
            for j in range(0, len(walls)):
                if i == j:
                    temp_1.append(100.0)
                    temp_2.append(100.0)
                    continue
                temp_1.append(getLengthBetween_A_B(
                    [walls[i][0], walls[i][1], walls[i][2]], 
                    [walls[j][0], walls[j][1], walls[j][2]]))
                temp_2.append(getLengthBetween_A_B(
                    [walls[i][0], walls[i][1], walls[i][2]], 
                    [walls[j][3], walls[j][4], walls[j][5]]))
                
            temp_1_list = list(zip(range(0, len(temp_1)), temp_1))
            temp_1_list_s = sorted(temp_1_list, key=lambda dist: dist[1])
            temp_2_list = list(zip(range(0, len(temp_2)), temp_2))
            temp_2_list_s = sorted(temp_2_list, key=lambda dist: dist[1])
            
            if min(temp_1) <= min(temp_2):
                swt = 0
                swt_2 = 0
                while True:
                    if temp_1_list_s[swt_2][0] in temp_index:
                        swt_2 = swt_2 + 1
                    else:
                        sorted_walls.append(walls[temp_1_list_s[swt_2][0]])
                        break
                temp_index.append(temp_1_list_s[swt_2][0])
            elif min(temp_1) > min(temp_2):
#                 index = temp_2.index(min(temp_2))
#                 sorted_walls.append(walls[index])
#                 temp_index.append(index)
                swt = 0
                swt_2 = 0
                while True:
                    if temp_2_list_s[swt_2][0] in temp_index:
                        swt_2 = swt_2 + 1
                    else:
                        sorted_walls.append(walls[temp_2_list_s[swt_2][0]])
                        break
                temp_index.append(temp_2_list_s[swt_2][0])
                
    pprint.pprint(temp_index)
    return sorted_walls

data_chain_sorted = wallsort_chain(data_sorted)

# pprint.pprint(data_chain_sorted)

data_variations_chain_sorted = []
for story_walls_data in getVariations(walls_stories_data):
    data_variations_chain_sorted.append(wallsort_chain(getData_nn(story_walls_data)))

print(len(data_variations_chain_sorted))
# pprint.pprint(data_variations_chain_sorted)

# for wall in data_sorted:
#     print(wallsort(wall))
# pprint.pprint(data_chain_sorted)
# pprint.pprint(data_sorted)

import json
with open('data.txt', 'w') as outfile:
    json.dump(data, outfile)

28
[9,
 5,
 8,
 3,
 2,
 0,
 16,
 7,
 24,
 13,
 10,
 17,
 14,
 11,
 19,
 4,
 12,
 20,
 21,
 6,
 22,
 15,
 25,
 27,
 18,
 23,
 26]
[0,
 15,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 27,
 17,
 11,
 14,
 13,
 12,
 1,
 16,
 20,
 18,
 21,
 10,
 26,
 22,
 23,
 24,
 25,
 9]
[3,
 15,
 16,
 5,
 4,
 19,
 10,
 9,
 8,
 27,
 17,
 2,
 14,
 13,
 12,
 1,
 11,
 20,
 6,
 21,
 22,
 23,
 26,
 25,
 24,
 7,
 18]
[0,
 15,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 27,
 17,
 11,
 14,
 13,
 12,
 1,
 16,
 20,
 18,
 21,
 10,
 26,
 22,
 23,
 24,
 25,
 9]
[3,
 15,
 16,
 5,
 4,
 19,
 10,
 9,
 8,
 27,
 17,
 2,
 14,
 13,
 12,
 1,
 11,
 20,
 6,
 21,
 22,
 23,
 26,
 25,
 24,
 7,
 18]
[0,
 15,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 27,
 17,
 11,
 14,
 13,
 12,
 1,
 16,
 20,
 18,
 21,
 10,
 26,
 22,
 23,
 24,
 25,
 9]
[3,
 15,
 16,
 5,
 4,
 19,
 10,
 9,
 8,
 27,
 17,
 2,
 14,
 13,
 12,
 1,
 11,
 20,
 6,
 21,
 22,
 23,
 26,
 25,
 24,
 7,
 18]
[0,
 15,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 27,
 17,
 11,
 14,
 13,
 12,
 1,
 16,
 20,
 18,
 21,
 10,
 26,
 22,
 23,
 24,
 2

D:\envir\anaconda3\envs\py_ifc_27\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars


## IfcRelContainedInSpatialStructure

In [56]:
structures = ifc_file.by_type("IfcRelContainedInSpatialStructure")
con_structures_info = []

con_sturctures_n = 0

for st in structures:
    relst = st.get_info()["RelatingStructure"]
    relele = [ relele_data.get_info() for relele_data in st.get_info()["RelatedElements"] ]
    con_structures_info.append(relele)

pprint.pprint(con_structures_info[:1])

[[{'Description': None,
   'GlobalId': u'2UZhzWc8v0fA0sfv__WgLn',
   'Name': u'Wand-002',
   'ObjectPlacement': #3005=IfcLocalPlacement(#2946,#3002),
   'ObjectType': None,
   'OwnerHistory': #29=IfcOwnerHistory(#28,#5,$,.NOCHANGE.,$,$,$,1215091190),
   'Representation': #3079=IfcProductDefinitionShape($,$,(#3039,#3073)),
   'Tag': u'DAC6A5FB-0527-4DD0-83-E9-B940B02D4AFA',
   'id': 3008,
   'type': 'IfcWallStandardCase'},
  {'Description': None,
   'GlobalId': u'3xvQAmf45C8PC3qSvtvqlg',
   'Name': u'Fenster-004',
   'ObjectPlacement': #3160=IfcLocalPlacement(#3124,#3157),
   'ObjectType': None,
   'OverallHeight': 1.5,
   'OverallWidth': 1.0,
   'OwnerHistory': #29=IfcOwnerHistory(#28,#5,$,.NOCHANGE.,$,$,$,1215091190),
   'Representation': #3899=IfcProductDefinitionShape($,$,(#4048)),
   'Tag': u'E2E8655F-6DD1-4336-A9-4B-13A89EA15E68',
   'id': 3903,
   'type': 'IfcWindow'},
  {'Description': None,
   'GlobalId': u'3fHoF9ddLDNg7p4qo0Wq7v',
   'Name': u'Fenster-004',
   'ObjectPlacement

#### 층의 건축요소 중에서 IfcWallStandardCase 에 대해, LocalPlacement 의 좌표를 받아온다

* 상황
 * 각 건축요소에 대해, PlacementRelTo와 RelativePlacement를 고려해서, World Cordinates로 변환해야 함
 * 위치 좌표는 RNN 계획에서 활용하지 않으며, FFDD에서 활용할 수 있음

* 문제
 * World Cordinates를 얻어야 함
 * 결국 얻는 것은 위치 좌표이기 때문에, RNN에서 활용할 수 없음
 * RNN 활용을 위해서는 다른 Parsing 과정이 필요함

In [66]:
structures = ifc_file.by_type("IfcRelContainedInSpatialStructure")
con_structures_info = []

con_sturctures_n = 0

for st in structures:
    relst = st.get_info()["RelatingStructure"]
    relele = [ relele_data.get_info()["ObjectPlacement"].get_info()["PlacementRelTo"].get_info()["PlacementRelTo"].get_info()["PlacementRelTo"].get_info()["RelativePlacement"].get_info() for relele_data in st.get_info()["RelatedElements"] if "IfcWallStandardCase" == relele_data.get_info()["type"]]
    
    con_structures_info.append(relele)

pprint.pprint(con_structures_info[:1])

[[{'Axis': #52=IfcDirection((0.,0.,1.)),
   'Location': #56=IfcCartesianPoint((0.,0.,0.)),
   'RefDirection': #44=IfcDirection((1.,0.,0.)),
   'id': 60,
   'type': 'IfcAxis2Placement3D'},
  {'Axis': #52=IfcDirection((0.,0.,1.)),
   'Location': #56=IfcCartesianPoint((0.,0.,0.)),
   'RefDirection': #44=IfcDirection((1.,0.,0.)),
   'id': 60,
   'type': 'IfcAxis2Placement3D'},
  {'Axis': #52=IfcDirection((0.,0.,1.)),
   'Location': #56=IfcCartesianPoint((0.,0.,0.)),
   'RefDirection': #44=IfcDirection((1.,0.,0.)),
   'id': 60,
   'type': 'IfcAxis2Placement3D'},
  {'Axis': #52=IfcDirection((0.,0.,1.)),
   'Location': #56=IfcCartesianPoint((0.,0.,0.)),
   'RefDirection': #44=IfcDirection((1.,0.,0.)),
   'id': 60,
   'type': 'IfcAxis2Placement3D'},
  {'Axis': #52=IfcDirection((0.,0.,1.)),
   'Location': #56=IfcCartesianPoint((0.,0.,0.)),
   'RefDirection': #44=IfcDirection((1.,0.,0.)),
   'id': 60,
   'type': 'IfcAxis2Placement3D'},
  {'Axis': #52=IfcDirection((0.,0.,1.)),
   'Location': #56=

- - - - - - -
이하, 기타 코드

In [43]:
help(ifc_file)

Help on file in module ifcopenshell.file object:

class file(__builtin__.object)
 |  Methods defined here:
 |  
 |  __getattr__(self, attr)
 |  
 |  __getitem__(self, key)
 |  
 |  __init__(self, f=None)
 |  
 |  __iter__(self)
 |  
 |  add(self, inst)
 |  
 |  by_guid(self, guid)
 |  
 |  by_id(self, id)
 |  
 |  by_type(self, type)
 |  
 |  create_entity(self, type, *args, **kwargs)
 |  
 |  get_inverse(self, inst)
 |  
 |  remove(self, inst)
 |  
 |  traverse(self, inst, max_levels=None)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [45]:
help(ifc_file.by_id)

Help on method by_id in module ifcopenshell.file:

by_id(self, id) method of ifcopenshell.file.file instance



In [46]:
help(ifcopenshell)

Help on package ifcopenshell:

NAME
    ifcopenshell

FILE
    d:\envir\anaconda3\envs\py_ifc_27\lib\site-packages\ifcopenshell\__init__.py

DESCRIPTION
    ###############################################################################
    #                                                                             #
    # This file is part of IfcOpenShell.                                          #
    #                                                                             #
    # IfcOpenShell is free software: you can redistribute it and/or modify        #
    # it under the terms of the Lesser GNU General Public License as published by #
    # the Free Software Foundation, either version 3.0 of the License, or         #
    # (at your option) any later version.                                         #
    #                                                                             #
    # IfcOpenShell is distributed in the hope that it will be useful,             #
    # b

In [50]:
help(ifcopenshell.ifcopenshell_wrapper)

Help on module ifcopenshell.ifcopenshell_wrapper in ifcopenshell:

NAME
    ifcopenshell.ifcopenshell_wrapper

FILE
    d:\envir\anaconda3\envs\py_ifc_27\lib\site-packages\ifcopenshell\ifcopenshell_wrapper.py

DESCRIPTION
    # This file was automatically generated by SWIG (http://www.swig.org).
    # Version 3.0.7
    #
    # Do not make changes to this file unless you know what you are doing--modify
    # the SWIG interface file instead.

CLASSES
    __builtin__.object
        IfcAbstractEntity
            HeaderEntity
                FileDescription
                FileName
                FileSchema
        IfcBaseEntity
            entity_instance
        IfcSpfHeader
        Material
        Representation
            BRep
            Serialization
            triangulation_double_precision
            triangulation_single_precision
        element_double_precision
            serialized_element_double_precision
            triangulation_element_double_precision
        element_s

In [51]:
help(ifcopenshell.entity_instance)

Help on class entity_instance in module ifcopenshell.entity_instance:

class entity_instance(__builtin__.object)
 |  Methods defined here:
 |  
 |  __dir__(self)
 |  
 |  __eq__(self, other)
 |  
 |  __getattr__(self, name)
 |  
 |  __getitem__(self, key)
 |  
 |  __hash__(self)
 |  
 |  __init__(self, e)
 |  
 |  __len__(self)
 |  
 |  __repr__(self)
 |  
 |  __setattr__(self, key, value)
 |  
 |  __setitem__(self, idx, value)
 |  
 |  attribute_name(self, attr_idx)
 |  
 |  attribute_type(self, attr)
 |  
 |  get_info(self)
 |  
 |  id(self)
 |  
 |  is_a(self, *args)
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  unwrap_value(v)
 |  
 |  walk(f, g, value)
 |  
 |  wrap_value(v)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [52]:
help(ifcopenshell.entity_instance.attribute_type)

Help on method attribute_type in module ifcopenshell.entity_instance:

attribute_type(self, attr) unbound ifcopenshell.entity_instance.entity_instance method



In [86]:
help(ifcopenshell.geom.create_shape)

Help on function create_shape in module ifcopenshell.geom.main:

create_shape(settings, inst, repr=None)

